In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('./preprocessing')
sys.path.append('./seq2seq')

In [3]:
from processor import Code_Intent_Pairs
from model import Seq2Seq
from data import get_train_loader, get_test_loader

### Define Hyperparameters

In [4]:
hyperP = {
    ## training parameters
    'batch_size' : 32,
    'lr' : 1e-3,
    'teacher_force_rate' : 0.90,
    'max_epochs' : 50,
    'lr_keep_rate' : 0.95,  # set to 1.0 to not decrease lr overtime
    'load_pretrain_code_embed': False,
    'freeze_embed': False,
    
    ## encoder architecture
    'encoder_layers' : 2,
    'encoder_embed_size' : 128,
    'encoder_hidden_size' : 384,
    'encoder_dropout_rate' : 0.3,
    
    ## decoder architecture
    'decoder_layers' : 2,
    'decoder_embed_size' : 128,
    'decoder_hidden_size' : 384,
    'decoder_dropout_rate' : 0.3,
    
    ## attn architecture
    'attn_hidden_size' : 384,
    
    ## visualization
    'print_every': 10,
}

### Load Data

In [5]:
code_intent_pair = Code_Intent_Pairs()

In [6]:
path = 'vocab/'
code_intent_pair.load_dict(path)
special_symbols = code_intent_pair.get_special_symbols()
word_size = code_intent_pair.get_word_size()
code_size = code_intent_pair.get_code_size()

In [7]:
train_path = 'processed_corpus/train.json'
train_entries = code_intent_pair.load_entries(train_path)
code_intent_pair.pad()

In [8]:
trainloader = get_train_loader(train_entries, special_symbols, hyperP)

In [9]:
valid_path = 'processed_corpus/valid.json'
valid_entries = code_intent_pair.load_entries(valid_path)
code_intent_pair.pad()

In [10]:
validloader = get_train_loader(valid_entries, special_symbols, hyperP)

In [11]:
test_path = 'processed_corpus/test.json'
test_entries = code_intent_pair.load_entries(test_path)

In [12]:
testloader = get_test_loader(test_entries)

### Define Model

In [13]:
model = Seq2Seq(word_size, code_size, hyperP)

In [14]:
import torch
if hyperP['load_pretrain_code_embed']:
    model.decoder.embed[0].load_state_dict(torch.load('./pretrain_code_lm/embedding-1556211835.t7'))
    if hyperP['freeze_embed']:
        for param in model.decoder.embed[0].parameters():
            param.requires_grad = False

In [15]:
# model = model.cuda()

In [16]:
# inp_seq, original_out_seq, padded_out_seq, out_lens = next(iter(trainloader))

In [17]:
# logits = model(inp_seq, padded_out_seq, out_lens)

### Training

In [18]:
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
optimizer = optim.Adam(model.parameters(), lr=hyperP['lr'], weight_decay = 1e-4)
loss_f = torch.nn.CrossEntropyLoss()

In [19]:
lr_keep_rate = hyperP['lr_keep_rate']
if lr_keep_rate != 1.0:
    lr_reduce_f = lambda epoch: lr_keep_rate ** epoch
    scheduler = LambdaLR(optimizer, lr_lambda=lr_reduce_f)

In [20]:
def train(model, trainloader, optimizer, loss_f, hyperP):
    model.train()
    total_loss = 0
    loss_sum = 0
    total_correct = 0
    size = 0
    print_every = hyperP['print_every']
    
    for i, (inp_seq, original_out_seq, padded_out_seq, out_lens) in enumerate(trainloader):
        logits = model(inp_seq, padded_out_seq, out_lens)
        loss = loss_f(logits, original_out_seq)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # show stats
        loss_sum += loss.item()
        total_loss += loss.item()
        _, predictions = torch.max(logits, dim=1)
        total_correct += (predictions == original_out_seq).sum()
        size += len(original_out_seq)

        if (i+1) % print_every == 0:
            print('Train: loss:{}\tacc:{}'.format(loss_sum/print_every, float(total_correct)/size), end='\r')
            loss_sum = 0
            total_correct = 0
            size = 0
    print()
    return total_loss / len(trainloader)

In [21]:
def valid(model, validloader, loss_f, hyperP):
    model.eval()
    old_rate = model.change_teacher_force_rate(0.0)
    loss_sum = 0
    total_correct = 0
    size = 0
    print_every = hyperP['print_every']
    
    with torch.no_grad():
        for i, (inp_seq, original_out_seq, padded_out_seq, out_lens) in enumerate(validloader):
            logits = model(inp_seq, padded_out_seq, out_lens)
            loss = loss_f(logits, original_out_seq)

            # show stats
            loss_sum += loss.item()
            _, predictions = torch.max(logits, dim=1)
            total_correct += (predictions == original_out_seq).sum()
            size += len(original_out_seq)

    print('Valid: loss:{}\tacc:{}'.format(loss_sum/len(validloader), float(total_correct)/size))
    model.change_teacher_force_rate(old_rate)
    return float(total_correct)/size

In [22]:
best_acc = 0.0

In [45]:
losses = []
for e in range(hyperP['max_epochs']):
    loss = train(model, trainloader, optimizer, loss_f, hyperP)
    losses.append(loss)
    acc = valid(model, validloader, loss_f, hyperP)
    if acc > best_acc:
        best_acc = acc
        model.save()
        print('model saved')
    if lr_keep_rate != 1.0:
        scheduler.step()

Train: loss:0.47406105101108553	acc:0.8642360278244183
Valid: loss:6.415027290582657	acc:0.20456354470681878
Train: loss:0.48295855820178984	acc:0.8697529383545215
Valid: loss:6.399059861898422	acc:0.2110639426903688
model saved
Train: loss:0.4157200872898102	acc:0.87742863996162156
Valid: loss:6.438829183578491	acc:0.21292119925709738
model saved
Train: loss:0.3895340710878372	acc:0.88990165507315965
Valid: loss:6.62345415353775	acc:0.21384982754046167
model saved
Train: loss:0.3683300971984863	acc:0.90333413288558411
Valid: loss:6.720470428466797	acc:0.21570708410719022
model saved


KeyboardInterrupt: 

In [46]:
model.load()

### Decoding

In [47]:
from decoder import Decoder, post_process_test, post_process, post_process_dummy
from evaluate import get_bleu_all, get_bleu_sent

In [48]:
beam_decoder = Decoder(model)

In [49]:
model.eval()
sos = special_symbols['code_sos']
eos = special_symbols['code_eos']
unk = special_symbols['code_unk']

In [50]:
dummy_code_list = []
true_code_list = []

for i, (src_seq, slot_map, code, intent) in enumerate(testloader):
    beams = beam_decoder.decode(src_seq, sos, eos, unk, beam_width=3)
    dummy_code =  post_process_dummy(slot_map, beams, code_intent_pair.idx2code)
    dummy_code_list.append(dummy_code)
    true_code_list.append(code)

In [51]:
get_bleu_all(dummy_code_list, true_code_list)

0.2686967398716274

### Evaluate Rerank

In [525]:
model.eval()
src_seq, slot_map, code, intent = testloader[61]

In [526]:
beams = beam_decoder.decode(src_seq, sos, eos, unk, beam_width=10)
post_process_test(intent, slot_map, beams, code_intent_pair.idx2code, code)

Insert directory 'apps' into directory `__file__`
sys.path.insert(1, os.path.join(os.path.dirname(__file__), 'apps'))
{'str_0': 'apps', 'var_0': '__file__'}
before process:
b_score:-0.44	score:0.22:	os . path . dirname ( __file__ , 'apps' )
b_score:-0.44	score:0.08:	os . path . dirname ( __file__ )
b_score:-0.43	score:0.16:	os . path . insert ( 'apps' , __file__ )
b_score:-0.40	score:0.24:	os = os . path . insert ( __file__ , 'apps' )
b_score:-0.40	score:0.28:	os . path . insert ( os . path . insert ( __file__ ) )
b_score:-0.36	score:0.41:	os . path . insert ( __file__ , os . path . dirname ( __file__ ) )
b_score:-0.35	score:0.20:	os . path . insert ( __file__ , 'apps' )
b_score:-0.29	score:0.07:	os . path . insert ( __file__ )
b_score:-0.28	score:0.20:	os . path . insert ( __file__ , 'apps' )
b_score:-0.24	score:0.07:	os . path . insert ( __file__ )
['insert', 'directory', '`', 'apps', '`', 'into', 'directory', '`', '__file__', '`']
after process:
b_score:-0.34	score:0.08:	os . path .

In [52]:
gen_code_list = []
dummy_code_list = []
true_code_list = []
model.eval()

for i, (src_seq, slot_map, code, intent) in enumerate(testloader):
    beams = beam_decoder.decode(src_seq, sos, eos, unk, beam_width=15)
    gen_code = post_process(intent, slot_map, beams, code_intent_pair.idx2code)
    gen_code_list.append(gen_code)
    true_code_list.append(code)

for i, (src_seq, slot_map, code, intent) in enumerate(testloader):
    beams = beam_decoder.decode(src_seq, sos, eos, unk, beam_width=15)
    dummy_code =  post_process_dummy(slot_map, beams, code_intent_pair.idx2code)
    dummy_code_list.append(dummy_code)
    

In [53]:
get_bleu_all(gen_code_list, true_code_list)

0.28847843830525005

In [54]:
get_bleu_all(dummy_code_list, true_code_list)

0.2739474443463464

In [55]:
from data import write_answer_json
write_answer_json(gen_code_list)